In [1]:
#importing the required modules
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os

In [4]:
#images folder path 
template_dir = "templates"

In [5]:
#load all the images
templates = {}
for filename in os.listdir(template_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        label = os.path.splitext(filename)[0]
        img = cv2.imread(os.path.join(template_dir, filename), cv2.IMREAD_GRAYSCALE)
        templates[label] = img

In [6]:
#function for matching the card with the template
def match_card(input_img):
    gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    best_score = -1
    best_label = "Unknown"

    for label, template in templates.items():
        template = cv2.resize(template, (gray.shape[1], gray.shape[0]))
        res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(res)

        if max_val > best_score:
            best_score = max_val
            best_label = label

    return best_label, best_score

In [7]:
#card detection GUI
class UNOCardGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("UNO Card Detection (Template Matching)")
        self.root.state("zoomed")
        self.root.configure(bg="#002147")

        #layout frames
        self.main_frame = tk.Frame(root, bg="#002147")
        self.main_frame.pack(fill="both", expand=True, padx=20, pady=20)

        self.left_frame = tk.Frame(self.main_frame, bg="#002147")
        self.left_frame.pack(side="left", fill="y", padx=20)

        self.right_frame = tk.Frame(self.main_frame, bg="#002147")
        self.right_frame.pack(side="right", fill="both", expand=True)

        
        #title and on screen instructions
        self.title_label = tk.Label(self.left_frame, text="UNO Card Detection", font=("Forte", 24, "bold"), fg="white", bg="#002147")
        self.title_label.pack(pady=20)

        self.label = tk.Label(self.left_frame, text="Upload a card to have it be detected:", font=("Aptos", 14), fg="white", bg="#002147")
        self.label.pack(pady=10)

        #upload buttons
        button_style = {"font":("Arial", 14), "fg":"black", "bg":"#B0C4DE", "width":30, "height":2, "bd":0, "relief":"raised"}
        self.upload_button = tk.Button(self.left_frame, text="Upload Card Image", command=self.upload_image, **button_style)
        self.upload_button.pack(pady=10)

        self.multi_button = tk.Button(self.left_frame, text="Upload Multiple Cards", command=self.upload_multiple_images, **button_style)
        self.multi_button.pack(pady=10)

        #canvas to display the uploaded card
        self.canvas = tk.Canvas(self.right_frame, width=300, height=500, bg="#002147", highlightbackground="white", highlightthickness=2)
        self.canvas.pack(pady=20)

        #result display
        self.result_label = tk.Label(self.left_frame, text="", font=("Arial", 16, "bold"), fg="white", bg="#002147")
        self.result_label.pack(pady=10)

        #clear button - will be hidden by default (shown once a card is uploaded)
        self.clear_button = tk.Button(self.right_frame, text="Clear", command=self.clear_canvas, font=("Arial", 12, "bold"), bg="#FF6B6B", fg="white", bd=0, relief="groove", padx=10, pady=5, cursor="hand2")
        self.clear_button.pack(pady=(40,20))
        self.clear_button.pack_forget()  # Hidden by default

        #store the uploaded images and results
        self.image_list = []
        self.result_list = []
        self.current_index = 0

        #creating navigation buttons (forward and backward) to navigate when multiple images are uploaded
        self.nav_frame = tk.Frame(self.right_frame, bg="#002147")
        self.nav_frame.pack(pady=5)
        nav_style = {"font":("Arial", 10, "bold"), "bg":"#4CAF50", "fg":"white", "bd":0, "relief":"groove", "padx":10, "pady":5, "cursor":"hand2"}
        self.prev_button = tk.Button(self.nav_frame, text="←", command=self.show_prev_image, **nav_style)
        self.next_button = tk.Button(self.nav_frame, text="→", command=self.show_next_image, **nav_style)
        self.prev_button.pack(side="left", padx=10)
        self.next_button.pack(side="right", padx=10)
        self.nav_frame.pack_forget()
        
    #uploading a card function
    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            img = cv2.imread(file_path)
            label, score = match_card(img)

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_pil = img_pil.resize((300, 500))
            img_tk = ImageTk.PhotoImage(img_pil)

            self.canvas.delete("all")
            self.canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
            self.canvas.image = img_tk
        
            self.result_label.config(text=f"Detected Card: {label} (Score: {score:.2f})")
            self.clear_button.pack()  #show the clear button

    #uploading multiple cards function
    def upload_multiple_images(self):
        file_paths = filedialog.askopenfilenames()
        if file_paths:
            self.image_list = []
            self.result_list = []
            self.current_index = 0

            for path in file_paths:
                img = cv2.imread(path)
                label, score = match_card(img)
                self.image_list.append(path)
                self.result_list.append(f"{os.path.basename(path)} → {label} (Score: {score:.2f})")

            self.show_image_by_index(0)
            self.nav_frame.pack()  #show navigation buttons
            self.clear_button.pack()  #show clear button

    #show the images from the stored list
    def show_image_by_index(self, index):
        img = cv2.imread(self.image_list[index])
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_pil = img_pil.resize((300, 500))
        img_tk = ImageTk.PhotoImage(img_pil)

        self.canvas.delete("all")
        self.canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
        self.canvas.image = img_tk

        self.result_label.config(text=self.result_list[index])

    #show the next image
    def show_next_image(self):
        if self.current_index + 1 < len(self.image_list):
            self.current_index += 1
            self.show_image_by_index(self.current_index)

    #show the previous image
    def show_prev_image(self):
        if self.current_index - 1 >= 0:
            self.current_index -= 1
            self.show_image_by_index(self.current_index)

    #clear the display and reset
    def clear_canvas(self):
        self.canvas.delete("all")
        self.result_label.config(text="")
        self.clear_button.pack_forget()
        self.nav_frame.pack_forget()
        self.image_list.clear()
        self.result_list.clear()
        self.current_index = 0

In [ ]:
#launching the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = UNOCardGUI(root)
    root.mainloop()